Imports

In [4]:
import json
from gensim import corpora, models, similarities
from nltk.tokenize import word_tokenize
import nltk
import os
import shutil
import numpy as np

ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\jopec\miniconda3\envs\projeto_final_ri\Lib\site-packages\scipy\linalg\__init__.py)

Carregando .jsons

In [ ]:
with open('map_documentos.json', 'r', encoding='utf-8') as f:
    map_documentos = json.load(f)
with open('map_consultas.json', 'r', encoding='utf-8') as f:
    map_consultas = json.load(f)

Download de dependências do NLTK
   

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

Indexar o Corpus com Vetorial (TF-IDF)

In [ ]:
print("Construindo o modelo Vetorial (TF-IDF)...")
doc_ids = list(map_documentos.keys())
doc_texts = list(map_documentos.values())
texts = [word_tokenize(text.lower()) for text in doc_texts]
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]
tfidf_model = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

# --- CORREÇÃO DE MEMORYERROR ---
# A classe 'MatrixSimilarity' tenta carregar toda a matriz na RAM.
# Para corpora grandes, usamos 'Similarity', que salva o índice em arquivos
# temporários no disco, consumindo muito menos memória.
temp_folder = './temp_similarity_index'
# Garante que a pasta temporária esteja limpa antes de usar
if os.path.exists(temp_folder):
    shutil.rmtree(temp_folder)
os.makedirs(temp_folder)
output_prefix = os.path.join(temp_folder, 'shard')
# A linha abaixo substitui 'similarities.MatrixSimilarity'
index = similarities.Similarity(output_prefix, tfidf_corpus, num_features=len(dictionary))
print("Modelo construído com sucesso!")

Construindo o modelo Vetorial (TF-IDF)...
Modelo construído com sucesso!


Função que retorna Top N documentos de uma consulta Vetorial

In [ ]:
def obter_top_n_vetorial(consulta_tokenizada, n=50):
    # 1. Converte a consulta para o espaço vetorial TF-IDF
    query_bow = dictionary.doc2bow(consulta_tokenizada)
    query_tfidf = tfidf_model[query_bow]

    # 2. Calcula os scores (similaridades) para cada documento no corpus
    doc_scores = index[query_tfidf]

    # 3. Pega os índices dos scores em ordem decrescente de pontuação
    top_n_indices = np.argsort(doc_scores)[::-1]

    # 4. Retorna apenas os 'n' primeiros IDs da lista ordenada
    return [int(doc_ids[idx]) for idx in top_n_indices[:n]]

Executar consulta e gerar os resultados top 50

In [ ]:
print("\nExecutando consulta de exemplo...")
# Escolha uma consulta para testar
id_consulta_exemplo = "1" # Usando a primeira consulta como exemplo
texto_consulta_exemplo = map_consultas[id_consulta_exemplo]

print(f"Texto da consulta (ID: {id_consulta_exemplo}): '{texto_consulta_exemplo}'")

# Tokeniza a consulta
consulta_tokenizada = texto_consulta_exemplo.lower().split()

# Top 50 documentos mais relevantes para a consulta
resultados_vetorial = obter_top_n_vetorial(consulta_tokenizada, n=50)

print(f"\nEncontrados {len(resultados_vetorial)} resultados.")


Executando consulta de exemplo...
Texto da consulta (ID: 182): 'mutations sonic hedgehog genes affect developmental disorders'

Encontrados 50 resultados.


Salvando resultados_vetorial como .json

In [ ]:
nome_arquivo_saida = 'resultados_vetorial.json'
print(f"\nSalvando resultados em '{nome_arquivo_saida}'...")

try:
    with open(nome_arquivo_saida, 'w', encoding='utf-8') as f:
        json.dump(
            resultados_vetorial,
            f,
            indent=4,
            ensure_ascii=False
        )
    print("Arquivo de resultados do modelo Vetorial salvo com sucesso!")
except Exception as e:
    print(f"ERRO ao salvar o arquivo JSON: {e}")
finally:
    # Limpa a pasta temporária do índice após o uso
    print("\nLimpando arquivos temporários do índice...")
    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)
        print("Limpeza concluída.")


Salvando resultados em 'resultados_vetorial.json'...
Arquivo de resultados do modelo Vetorial salvo com sucesso!

Limpando arquivos temporários do índice...
Limpeza concluída.
